In [2]:
import pandas as pd
import numpy as np
import re
import datetime
pd.options.display.max_columns = None

In [4]:
df = pd.read_csv('../datos/historico_tickets.csv', index_col = 0)
df.sample(3)

,id_ticket,fecha,id_empleado,id_agente,categoría,tipo,severidad,prioridad,días_resolución,satisfacción,semestre,año
15091,SWLTSR-6643171106,2018-03-12,1440,1,Sistema,Solicitud,2 - Normal,1 - Baja,0,4,1 semestre,2018
6170,KHLTSR-7542408532,2016-02-08,873,35,Sistema,Solicitud,2 - Normal,3 - Alta,5,3,1 semestre,2016
13117,THLTER-1442970997,2017-08-23,709,29,Software,Solicitud,2 - Normal,3 - Alta,4,4,2º semestre,2017


Ejercicio nº1 Cread una columna nueva y una función que nos de el mes en el que se solicitó el servicio. Tened en cuenta que todas las filas tienen la misma estructura. A la hora de escribir la función debemos tener en cuenta que puede que no haya la información de mes en la columna de la fecha. En ese caso devolved un nulo (NaN).
📌 Nota La fecha tendrá que ir formato de string como "Enero", "Febrero", etc.
💡 Pista 💡Podréis usar regex, pero no es estrictamente necesario.

In [5]:
df['fecha'] = df['fecha'].apply(pd.to_datetime)

In [6]:
def sacar_mes(f):
    return f.strftime("%B")

In [7]:
df['mes'] = df["fecha"].apply(sacar_mes)

In [8]:
df.sample(3)

,id_ticket,fecha,id_empleado,id_agente,categoría,tipo,severidad,prioridad,días_resolución,satisfacción,semestre,año,mes
12837,KHLTNT-1043979423,2020-05-28,1472,24,Acceso/Login,Solicitud,2 - Normal,3 - Alta,0,4,1 semestre,2020,May
19554,SHLTET-4543993399,2020-06-11,839,23,Software,Solicitud,2 - Normal,3 - Alta,5,4,1 semestre,2020,June
13162,KHLTSR-2743902637,2020-03-12,553,16,Sistema,Solicitud,2 - Normal,3 - Alta,7,5,1 semestre,2020,March


Ejercicio nº2 En la tabla de empleados, tenemos los nombres y los apellidos de los empleados, pero en columnas separadas. En los ejercicios de pair de la lección del groupby vimos que si quisieramos agrupar por los nombres, esto se nos puede complicar. Ahora nos piden que creemos una columna donde tengamos el nombre y el apellido juntos.
📌 NOTA La columna nueva debe tener el nombre y apellido con la primera letra en mayúsculas, por ejemplo "Jesús Villa".
📌 NOTA Una vez hallamos creado la columna nueva, eliminad las columnas de nombre y apellido originales.

In [10]:
dfe = pd.read_csv('../datos/empleados_puestos.csv', index_col = 0)
dfe.sample(3)

,Turno,ID Empleado,Apellido,Nombre,ID Puesto,Email,Puesto,Departamento,Tipo Puesto
80,NaN,95,PEREZ,IGNACIO,15-24100,ignacio.perez@tractchun.com,Supervisor de Producción,PRODUCCION,Mandos Medios
1767,NaN,1104,FIMBRES,BRACAMONTE,70-21100,bracamonte.fimbres@tractchun.com,Supervisor de Recursos Humanos,Recursos Humanos,Mandos Medios
865,NaN,595,ISLAS,ALICIA,70-21110,alicia.islas@tractchun.com,Entrenamiento,Recursos Humanos,Administrativo


In [11]:
def nombre_completo(nombre, apellido):
    try:
        nc = nombre.title() + ' ' + apellido.title()
        return nc
    except:
        return np.nan

In [12]:
dfe['Nom_Completo'] = dfe.apply(lambda dataf: nombre_completo(dataf['Nombre'], dataf['Apellido']), axis = 1)

In [13]:
dfe.drop('Apellido', axis = 1, inplace = True)
dfe.drop('Nombre', axis = 1, inplace = True)

In [14]:

dfe.sample(3)

,Turno,ID Empleado,ID Puesto,Email,Puesto,Departamento,Tipo Puesto,Nom_Completo
1499,NaN,1556,90-14100,enrique.cortes@tractchun.com,Ingeniero de Calidad,INGENIERIA DE CALIDAD,Operativo,Enrique Cortes
214,NaN,1694,15-24100,heriberto.romero@tractchun.com,Supervisor de Producción,PRODUCCION,Mandos Medios,Heriberto Romero
251,NaN,1547,15-24100,leonor.torrico@tractchun.com,Supervisor de Producción,PRODUCCION,Mandos Medios,Leonor Torrico


Ejercicio nª3 En el pair programming desde a dirección nos pedían que agruparamos los datos en base a la columna de "categoría", puede que en aquel momento nos dieramos cuenta que había varías categorías que eran igual (en concreto "Acceso Login") pero que en algunos casos tenía unos símbolos raros (&, %, -). Es el momento de limpiar esta columna. El objetivo, reemplazar todos estos símbolos raros por "/", para conseguir homogeneizar los valores de esta columna. Para ellos deberemos crear una función.

In [15]:
df.sample(2)

,id_ticket,fecha,id_empleado,id_agente,categoría,tipo,severidad,prioridad,días_resolución,satisfacción,semestre,año,mes
9988,KMLTSR-5543237815,2018-05-17,1931,8,Sistema,Solicitud,2 - Normal,2 - Media,5,4,1 semestre,2018,May
4283,GHRTSR-1643684792,2019-08-07,1699,17,Sistema,Solicitud,3 - Mayor,3 - Alta,6,5,2º semestre,2019,August


In [16]:
categorías = df['categoría'].unique()
categorías

array(['Acceso&%Login', 'Sistema', 'Software', 'Hardware',
       'Acceso%&Login', 'Acceso-&Login', 'Acceso99Login', 'Acceso/Login'],
      dtype=object)

In [17]:
def limpiar_cat(c):
    if c == 'Acceso&%Login' or c == 'Acceso%&Login' or c == 'Acceso-&Login' or c == 'Acceso99Login':
        return 'Acceso/Login'
    else:
        return c

In [19]:
df['categoría'] = df['categoría'].apply(limpiar_cat)

In [20]:
df['categoría'].unique()

array(['Acceso/Login', 'Sistema', 'Software', 'Hardware'], dtype=object)

Ejercicio nª4 Una vez que hayáis terminado eliminad las columnas originales conservando las nuevas.

In [21]:
df.sample(4)

,id_ticket,fecha,id_empleado,id_agente,categoría,tipo,severidad,prioridad,días_resolución,satisfacción,semestre,año,mes
867,GDLTSR-0542887752,2017-06-01,1465,17,Sistema,Solicitud,2 - Normal,0 - Sin Asignar,5,5,1 semestre,2017,June
4380,KDLTNR-3042646747,2016-10-03,1754,27,Acceso/Login,Solicitud,2 - Normal,0 - Sin Asignar,0,3,2º semestre,2016,October
17139,THLTNT-4043387889,2018-10-14,1228,38,Acceso/Login,Solicitud,2 - Normal,3 - Alta,0,4,2º semestre,2018,October
13094,SHLTNT-0043288188,2018-07-07,288,11,Acceso/Login,Solicitud,2 - Normal,3 - Alta,0,4,2º semestre,2018,July


Ejercicio nª5 Guardad el dataframe en un csv que usaremos más adelante

In [22]:
df.to_csv("../datos/tickets_final.csv")
dfe.to_csv("../datos/empleados_final.csv")